# Diving deep into scattering problems with Kwant

Slides adapted from https://github.com/jbweston/maryland-kwant-tutorial

<br/>
<footer>Joseph Weston – CJC 2019</footer>


# The Plan

*I will assume that you are already familiar with Kwant*

+ Define the 2 main equations that Kwant uses to solve the scattering problem
    - Generalized eigenvalue problem for the lead modes
    - Linear system (involving the lead modes) for the actual scattering matrix calculation
+ Look closely at how Kwant calculates the lead modes
    - Stabilization when the inter-cell hopping is singular
    - Use of a Schur vector basis for the evanescent modes
    - Use of symmetries

<div style="max-width: 60%; float: left">
    <h1> An illustrative example </h1>
<ul >
    <li> Quasi-1D system with electron/hole and spin bands</li>
    <li> Bernevig-Hughes-Zhang (BHZ) model </li>
    <li> Quantum S pin Hall phase </li>
</ul>
    $$\begin{align}
H(k_x,& k_y) =\\
    & + μ(σ_0⊗σ_0)\\
    & + M(σ_0⊗σ_z)\\
    & - B(k_x^2 + k_y^2)(σ_0⊗σ_z)- D(k_x^2 + k_y^2)(σ_0⊗σ_0)\\
    & + A[k_x(σ_z⊗σ_x) + k_y(σ_0⊗σ_y)]\\
\end{align}$$

basis: $\big(|↑\rangle, |↓\rangle\big)⊗\big(|e\rangle, |h\rangle\big)$
</div>
<img src="images/system-sketch.svg" width="35%" style="float: right; display: inline"> </img>

In [ ]:
# So our matrices look nice
import sympy
sympy.init_printing()

# So we can plot things
%matplotlib notebook
from matplotlib import pyplot as plt

# general numerics
import numpy as np

# useful definitions for later
sigma_0 = np.array([[1, 0], [0, 1]])
sigma_x = np.array([[0, 1], [1, 0]])
sigma_y = np.array([[0, -1j], [1j, 0]])
sigma_z = np.array([[1, 0], [0, -1]])

In [ ]:
import kwant.continuum

bhz_continuum = '''
    + mu * kron(sigma_0, sigma_0)
    + M * kron(sigma_0, sigma_z)
    - B * (k_x**2 + k_y**2) * kron(sigma_0, sigma_z) - D * (k_x**2 + k_y**2) * kron(sigma_0, sigma_0)
    + A * (k_x * kron(sigma_z, sigma_x) + k_y * kron(sigma_0, sigma_y))
'''

bhz_model = kwant.continuum.discretize(bhz_continuum)

kwant.continuum.sympify(bhz_continuum)  # just to see what it looks like

In [ ]:
syst = kwant.Builder() # We'll fill this with the TB model of our scattering region

def scattering_region(site):
    x, y = site.tag  # position in integer lattice coordinates
    y = y - 50
    return (30**2 < (x**2 + y**2) < 50**2  # inside a ring
            and (x >= 0 and y <= 0))  # lower-right quadrant ofring

# Fill our scattering region with the BHZ model in a certain region of space
syst.fill(template=bhz_model, shape=scattering_region, start=(0, 10))

kwant.plot(syst);

In [ ]:
lead_x = kwant.Builder(symmetry=kwant.TranslationalSymmetry((-1, 0)),
                       time_reversal=np.kron(1j * sigma_y, sigma_0),
                       conservation_law=np.kron(-sigma_z, sigma_0),  # spin conservation
                      )

lead_x.fill(bhz_model, lambda site: 1 <= site.tag[1] <= 20, (0, 1))

kwant.plot(lead_x);

In [ ]:
# Just the same as the x-direction, so we skip this in the slides
lead_y = kwant.Builder(symmetry=kwant.TranslationalSymmetry((0, 1)),
                       time_reversal=np.kron(1j * sigma_y, sigma_0),
                       conservation_law=np.kron(-sigma_z, sigma_0),
                      )

lead_y.fill(bhz_model, lambda site: 30 <= site.tag[0] <= 49, (35, 0));

In [ ]:
syst.attach_lead(lead_x)
syst.attach_lead(lead_y)

kwant.plot(syst);

In [ ]:
fsyst = syst.finalized()  # Transform the system into an efficient form for numerics
print(fsyst)

# What did we just do?

*We defined the Hamiltonian of our system as an "infinite matrix"*

<div>
<img src="images/curve-system.png" width=30% style="float: left"></img>
<div style="margin-right: 10%; float:right">
$$
H = 
\begin{pmatrix}
H_s & P^T_s V_l^\dagger & \\
V_lP_s & H_l               & V_l^\dagger \\
    & V_l               & H_l & V_l^\dagger \\
    &                   &     &       & \ddots
\end{pmatrix}
$$
    
$$\begin{align}
&H_s: \text{Scattering region Hamiltonian} \\
&H_l: \text{Lead(s) unit cell Hamiltonian} \\
&V_l: \text{Lead(s) inter-cell hopping Hamiltonian} \\
\end{align}$$
</div>
</div>

# Solving Schrödinger equation for an ∞ tight-binding model

1. Solve Schrödinger equation in the leads *first* (using Bloch's theorem)
2. Match the solution in the scattering region

# Solving the Schrodinger equation in the leads

$$
\begin{pmatrix}
   \ddots &                   &     &       & \\
V_l & H_l & V_l^\dagger \\
    & V_l               & H_l & V_l^\dagger \\
    &                   & V_l               & H_l & V_l^\dagger \\
    &                   &     &       & \ddots \\
\end{pmatrix}
\begin{pmatrix}
⋮ \\ ψ_{-1} \\ ψ_0 \\ ψ_1 \\ ⋮ 
\end{pmatrix}
=
E
\begin{pmatrix}
⋮ \\ ψ_{-1} \\ ψ_0 \\ ψ_1 \\ ⋮ 
\end{pmatrix}
$$

Use Bloch's theorem: $ψ_n = λ^n φ$

$$
\begin{pmatrix}
   \ddots &                   &     &       & \\
V_l & H_l & V_l^\dagger \\
    & V_l               & H_l & V_l^\dagger \\
    &                   & V_l               & H_l & V_l^\dagger \\
    &                   &     &       & \ddots \\
\end{pmatrix}
\begin{pmatrix}
⋮ \\ λ^{-1}φ \\ φ \\ λφ \\ ⋮ 
\end{pmatrix}
=
E
\begin{pmatrix}
⋮ \\ λ^{-1}φ \\ φ \\ λφ \\ ⋮ 
\end{pmatrix}
$$

Each row reduces to the following:

$$
V_l φ + λ(H_l - E)φ + λ^2V_l^\daggerφ = 0
$$

A finite problem that we can solve!

*Given* $E$, what are the corresponding $φ$'s and $λ$'s?

**Quadratic eigenvalue problem in λ: what do?**

write: $ξ \equiv λφ$, then

<br/>

$$
\begin{pmatrix}
H_l - E & V_l^\dagger \\ 1 & 0
\end{pmatrix}
\begin{pmatrix}
φ \\ ξ
\end{pmatrix}
=
λ^{-1}
\begin{pmatrix}
-V_l & 0 \\ 0 & 1
\end{pmatrix}
\begin{pmatrix}
φ \\ ξ
\end{pmatrix}
$$

<br/>

→ Generalized Eigenvalue Problem.

<br/>

*propagating* modes: $|λ| = 1$ ; *evanescent* modes: $|λ| \ne 1$

*mode velocity*: $v = iφ^\dagger[λV_l^\dagger - λ^{-1}V_l]φ$

$φ$'s then *normalized* to have unit velocity (current)

Arbitrary state in unit cell $n$:

$$\hat{φ}_n = ΦΛ^nq$$

$$\begin{align}
Φ: &\text{matrix of mode wavefunctions ($φ$'s)} \\
Λ: &\text{diagonal matrix of eigenvalues} \\
q: &\text{column vector of mode amplitudes} \\
\end{align}$$

# Solving the scattering problem

We still need to solve our original Schrödinger equation:

$$
\begin{pmatrix}
H_s & P^T_s V_l^\dagger & \\
V_lP_s & H_l               & V_l^\dagger \\
    & V_l               & H_l & V_l^\dagger \\
    &                   &     &       & \ddots
\end{pmatrix}
\begin{pmatrix}
ψ_s \\ ψ_1 \\ ψ_2 \\ \vdots
\end{pmatrix}
=
E
\begin{pmatrix}
ψ_s \\ ψ_1 \\ ψ_2 \\ \vdots
\end{pmatrix}
$$

Sub in our mode decomposition in the leads

($Φ_{+}$: *outgoing* modes, $Φ_{p-}$ : *incoming propagating* modes)

$$
\begin{pmatrix}
(H_s - E) & P^T_s V_l^\dagger & \\
V_lP_s & (H_l - E)           & V_l^\dagger \\
    & V_l               & (H_l - E) & V_l^\dagger \\
    &                   &     &       & \ddots
\end{pmatrix}
\begin{pmatrix}
ψ_s \\ Φ_{+}Λ_{+}q_{+} + Φ_{p-}Λ_{p-}q_{p-} \\ Φ_{+}(Λ_{+})^2q_{+} + Φ_{p-}(Λ_{p-})^2q_{p-} \\\vdots
\end{pmatrix}
=
0
$$

Rows 3 and onwards are trivially zero → reduces the problem to a finite one

$$
\begin{pmatrix}
(H_s - E) & P^T_s V_l^\dagger Φ_+ Λ_+ & P^T_s V_l^\dagger Φ_{p-} Λ_{p-}\\
V_lP_s & -V_l Φ_+ & -V_l Φ_{p-}\\
\end{pmatrix}
\begin{pmatrix}
ψ_s \\ q_{+} \\ q_{p-}
\end{pmatrix}
=
0
$$

Finding all energy $E$ solutions → finding null space of left-hand side

Ideally we would like a "nice" basis that will make it easy to do physics later

Leads individually in thermal equilibrium → choose basis where *incoming mode occupation* is a good quantum number


Set $q_{p-} = \begin{pmatrix} 1, & 0, & …\end{pmatrix}^T$ so that $Φ_{p-}q_{p-} = φ_{-,0}$ and $Φ_{p-}Λ_{p-}q_{p-} = λ_{-,0}φ_{-,0}$

$$
\begin{pmatrix}
(H_s - E) & P^T_s V_l^\dagger Φ_+ Λ_+ \\
V_lP_s & -V_l Φ_+\\
\end{pmatrix}
\begin{pmatrix}
ψ_s \\ q_{+}
\end{pmatrix}
=
\begin{pmatrix}
-P^T_s V_l^\dagger φ_{-,0}\,λ_{-,0} \\
V_l φ_{-,0}
\end{pmatrix}
$$

Fully determined linear system to solve

A set of such linear systems: each injecting particles in a different incoming propagating mode

Stack them together:


$$
\begin{pmatrix}
(H_s - E) & P^T_s V_l^\dagger Φ_+ Λ_+ \\
V_lP_s & -V_l Φ_+\\
\end{pmatrix}
\begin{pmatrix}
Ψ_s \\ Q_{+}
\end{pmatrix}
=
\begin{pmatrix}
-P^T_s V_l^\dagger Φ_{p-}Λ_{p-} \\
V_l Φ_{p-}
\end{pmatrix}
$$

$Ψ_s$: matrix of **scattering wavefunctions** in the scattering region

$Q_+$: **extended scattering matrix**, outgoing (propagating and evanescent) mode amplitudes

$S$:   **scattering matrix**, submatrix of $Q_+$ that concerns the *propagating modes only*

# The equations to solve

$$
\begin{pmatrix}
H_l - E & V_l^\dagger \\ 1 & 0
\end{pmatrix}
\begin{pmatrix}
φ \\ ξ
\end{pmatrix}
=
λ^{-1}
\begin{pmatrix}
-V_l & 0 \\ 0 & 1
\end{pmatrix}
\begin{pmatrix}
φ \\ ξ
\end{pmatrix}
$$

<br/>

$$
\begin{pmatrix}
(H_s - E) & P^T_s V_l^\dagger Φ_+ Λ_+ \\
V_lP_s & -V_l Φ_+\\
\end{pmatrix}
\begin{pmatrix}
Ψ_s \\ Q_{+}
\end{pmatrix}
=
\begin{pmatrix}
-P^T_s V_l^\dagger Φ_{p-}Λ_{p-} \\
V_l Φ_{p-}
\end{pmatrix}
$$


# The mode problem

$$
\begin{pmatrix}
H_l - E & V_l^\dagger \\ 1 & 0
\end{pmatrix}
\begin{pmatrix}
φ \\ ξ
\end{pmatrix}
=
λ^{-1}
\begin{pmatrix}
-V_l & 0 \\ 0 & 1
\end{pmatrix}
\begin{pmatrix}
φ \\ ξ
\end{pmatrix}
$$

## Case: $V_l$ is invertible

multiply by the inverse of the RHS:

$$
\begin{pmatrix}
-V_l^{-1}(H_l - E) & -V_l^{-1}V_l^\dagger \\ 1 & 0
\end{pmatrix}
\begin{pmatrix}
φ \\ ξ
\end{pmatrix}
=
λ^{-1}
\begin{pmatrix}
φ \\ ξ
\end{pmatrix}
$$

Define

$
    V_l = AB^\dagger
    \quad\quad A = \frac{V_l}{\sqrt{\lVert V_l\rVert}}
    \quad\quad B = \sqrt{\lVert V_l\rVert}
    \quad\quad \bar{φ} = B^\dagger φ
    \quad\quad \bar{ξ} = A^\dagger ξ
$

Then we write the problem as

$$
\begin{pmatrix}
-A^{-1}(H_l - E)(B^\dagger)^{-1} & -A^{-1}B \\
A^\dagger(B^\dagger)^{-1} & 0
\end{pmatrix}
\begin{pmatrix}
\bar{φ} \\ \bar{ξ}
\end{pmatrix}
=
λ^{-1}
\begin{pmatrix}
\bar{φ} \\ \bar{ξ}
\end{pmatrix}
$$

**this** is the problem Kwant solves


<img src="images/kwant/modes-nonsingular.png" width=50% style="float: left;"/>

<div style="float: left; max-width=0%; padding-left: 5%">
$$
\begin{pmatrix}
-A^{-1}(H_l - E)(B^\dagger)^{-1} & -A^{-1}B \\
A^\dagger(B^\dagger)^{-1} & 0
\end{pmatrix}
$$

[here's the code](https://gitlab.kwant-project.org/kwant/kwant/blob/master/kwant/physics/leads.py#L270)
</div>

## Case: $V_l$ is not invertible

Define:

$
    V_l = USW^\dagger
    \quad\quad A = U \sqrt{S}
    \quad\quad B = W \sqrt{S}
    \quad\quad \bar{φ} = B^\dagger φ
    \quad\quad \bar{ξ} = A^\dagger ξ
$

The first line of the GEVP is

$$
(H_l - E)φ = - B\bar{ξ} -λ^{-1}A\bar{φ}
$$

Now we add $i AA^\daggerφ + i BB^\daggerφ$ to both sides and invert the matrix on the LHS:

$$
φ = 
    \underbrace{[H_l - E + iAA^\dagger + iBB^\dagger]^{-1}}_{\bar{H_l}^{-1}}
    (-B\bar{ξ} -λ^{-1}A\bar{φ} + iλ^{-1}A\bar{ξ} + iB\bar{φ})
$$

Substitute this into the original GEVP and we get:

$$
\begin{pmatrix}
-iA^\dagger \bar{H_l}^{-1} B & A^\dagger \bar{H_l}^{-1} B \\
1 - iB^\dagger \bar{H_l}^{-1} B & B^\dagger \bar{H_l}^{-1} B
\end{pmatrix}
\begin{pmatrix}
\bar{φ} \\ \bar{ξ}
\end{pmatrix}
=
λ^{-1}
\begin{pmatrix}
-A^\dagger \bar{H_l}^{-1} A & iA^\dagger \bar{H_l}^{-1} A - 1 \\
-B^\dagger \bar{H_l}^{-1} A & iB^\dagger \bar{H_l}^{-1} A
\end{pmatrix}
\begin{pmatrix}
\bar{φ} \\ \bar{ξ}
\end{pmatrix}
$$

If the RHS matrix is invertible then we apply it's inverse to the above equation, otherwise we leave it as a GEVP.

<img src="images/kwant/modes-singular.png" width=50% style="float: left;"/>

<div style="float: left; max-width=0%; padding-left: 5%">

$A$ → `u`

$B$ → `v`

$\bar{H_l}^{-1} B$ → `kla.lu_solve(sol, v)`

$\bar{H_l}^{-1} A$ → `kla.lu_solve(sol, u)`

$$
\begin{pmatrix}
-iA^\dagger \bar{H_l}^{-1} B & A^\dagger \bar{H_l}^{-1} B \\
1 - iB^\dagger \bar{H_l}^{-1} B & B^\dagger \bar{H_l}^{-1} B
\end{pmatrix}
$$
    
$$
\begin{pmatrix}
-A^\dagger \bar{H_l}^{-1} A & iA^\dagger \bar{H_l}^{-1} A - 1 \\
-B^\dagger \bar{H_l}^{-1} A & iB^\dagger \bar{H_l}^{-1} A
\end{pmatrix}
$$

[Here's the code](https://gitlab.kwant-project.org/kwant/kwant/blob/master/kwant/physics/leads.py#L361)
</div>

# Solving the GEVP

Once we've built the appropriate eigenvalue or generalized eigenvalue problem we then need to solve it.

The eigenproblem is not Hermitian, so the eigenvectors are typically not orthogonal.

Instead we perform a (generalized) *Schur decomposition*.

We need the actual eigenvectors for the propagating modes (so that the scattering matrix elements are meaningful), but we can leave the evanescent subspace as Schur vectors.

<img src="images/kwant/modes-wavefunction-extraction.png" width=50% style="float: left;"/>

<div style="float: left; padding-left: 5%">
    
[Here's the code](https://gitlab.kwant-project.org/kwant/kwant/blob/master/kwant/physics/leads.py#L883)

</div>

# Massaging the modes

Post-processing of the propagating modes is still required

+ modes with identical momentum need to be orthogonalized
+ symmetries should transform modes into one another where appropriate
  (e.g. time-reversal maps k → -k)

<img src="images/kwant/modes-assembly.png" width=50% style="float: left;"/>

<div style="max-width: 50%; float: left; padding-left: 5%">
    
[Here's the code](https://gitlab.kwant-project.org/kwant/kwant/blob/master/kwant/physics/leads.py#L900)

[Here's `make_proper_modes`](https://gitlab.kwant-project.org/kwant/kwant/blob/master/kwant/physics/leads.py#L616)

`wavefunctions` → propagating modes in the tight-binding basis

`vecs`($\bar{φ}$) and `vecslmbdainv` ($\bar{ξ}$) → *all* modes in the hopping SVD basis.

These are used for solving the scattering problem

</div>

# The scattering problem

$$
\begin{pmatrix}
(H_s - E) & P^T_s V_l^\dagger Φ_+ Λ_+ \\
V_lP_s & -V_l Φ_+\\
\end{pmatrix}
\begin{pmatrix}
Ψ_s \\ Q_{+}
\end{pmatrix}
=
\begin{pmatrix}
-P^T_s V_l^\dagger Φ_{p-}Λ_{p-} \\
V_l Φ_{p-}
\end{pmatrix}
$$


Uses the stabilized modes of the leads. Rewrite using $V_l = AB^\dagger$:

$$
\begin{pmatrix}
(H_s - E) & P^T_s BA^\dagger Φ_+ Λ_+ \\
AB^\dagger P_s & -AB^\dagger Φ_+\\
\end{pmatrix}
\begin{pmatrix}
Ψ_s \\ Q_{+}
\end{pmatrix}
=
\begin{pmatrix}
-P^T_s BA^\dagger Φ_{p-}Λ_{p-} \\
AB^\dagger Φ_{p-}
\end{pmatrix}
$$


recall that $\bar{Ξ} = A^\dagger Φ Λ$ and $\bar{Φ} = B^\dagger Φ$

$$
\begin{pmatrix}
(H_s - E) & P^T_s B\,\bar{Ξ}_+ \\
AB^\dagger P_s & -A\,\bar{Φ}_+\\
\end{pmatrix}
\begin{pmatrix}
Ψ_s \\ Q_{+}
\end{pmatrix}
=
\begin{pmatrix}
-P^T_s B\,\bar{Ξ}_{p-} \\
A\,\bar{Φ}_{p-}
\end{pmatrix}
$$


$A$ is injective, so it has a left inverse; apply this to the second row:

$$
\begin{pmatrix}
(H_s - E) & P^T_s B\,\bar{Ξ}_+ \\
B^\dagger P_s & -\bar{Φ}_+\\
\end{pmatrix}
\begin{pmatrix}
Ψ_s \\ Q_{+}
\end{pmatrix}
=
\begin{pmatrix}
-P^T_s B\,\bar{Ξ}_{p-} \\
\bar{Φ}_{p-}
\end{pmatrix}
$$

<div style="float: left; max-width=40%">
    <img src="images/kwant/scattering-udef.png" width=30%/>
    <img src="images/kwant/scattering-uoutdef.png" width=30%/>
    <img src="images/kwant/scattering-linsys.png" width=30%/>
    <img src="images/kwant/scattering-linsys-input.png" width=30%/>
</div>

<div style="max-width: 50%; float: left; padding-left: 5%;">
    
[Here's the code](https://gitlab.kwant-project.org/kwant/kwant/blob/master/kwant/solvers/common.py#L223)

$\bar{Φ}$ → `ulinv`

$\bar{Ξ}$ → `u`

$B$ → `svd_v`

$P$ → `transf`



$$
\begin{pmatrix}
(H_s - E) & P^T_s B\,\bar{Ξ}_+ \\
B^\dagger P_s & -\bar{Φ}_+\\
\end{pmatrix}
$$
$$
\begin{pmatrix}
-P^T_s B\,\bar{Ξ}_{p-} \\
\bar{Φ}_{p-}
\end{pmatrix}
$$
    
</div>